# Identifying Image Features Using PCA

A variety of applications, including face recognition, computer vision, and medical imaging, are based on performing computations on image data. These image data are high-dimensional, which it makes it difficult to store large volumes of images as well as extract patterns from images. A key step in image processing algorithms is to identify low-dimensional features, which can then be used to process, manipulate, and compress/store the images. Principal component analysis is an important tool for doing so.

Analysis of images is a special case of PCA. Let $x_{1},\ldots,x_{N}$ denote a set of images, where $x_{i} \in \mathbb{R}^{M}$ and $M$ is the total number of pixels in each image. The data matrix $X$ is an $M \times N$ matrix in which the columns are $x_{1},\ldots,x_{N}$. By taking the SVD of $X$ and performing PCA, we can identify the principal components of the images, which we can then use to compress and classify them.

In this project, you will work with the Extended Yale B dataset (unpadded), which is also known as the Eigenfaces dataset \cite{gross2005face}. The dataset consists of images of the faces of 15 people. There are multiple images for each person, representing different facial expressions and lighting conditions. 

In [ ]:
%pip install imageio

In [37]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

## Part 1 
Download the image dataset. Load all of the images for the first 13 subjects. Your training dataset will consist of these images.

In [ ]:
# Part 1: Loading the data
import os
suffixes = ['centerlight', 'glasses', 'happy', 'leftlight', 'noglasses', 
            'normal', 'rightlight', 'sad', 'sleepy', 'surprised', 'wink']

num_faces_training = 13
num_suffixes = len(suffixes)
num_imgs_total = num_faces_training * num_suffixes

img_height = 116
img_width = 98
img_size = img_height * img_width

img_matrix = np.zeros((img_size, num_imgs_total))

# *************************
# Implement your code here
# 
# *************************

In [ ]:
# Check if the total number of images matches the expected number
if img_matrix.shape == (img_size, num_imgs_total):
    print("Part 1: Pass - Data loaded successfully with correct dimensions.")
else:
    print("Part 1: Fail - Data dimensions or loading process may have issues.")

## Part 2: Generate X matrix
Generate the data matrix $X$ for the images in your training dataset. 

In [ ]:
# *************************
# Implement your code here
# 
# feature_mean = 
# X =
# *************************

import matplotlib.pyplot as plt

plt.imshow(feature_mean.reshape(img_height, img_width), cmap='gray')
plt.title("Mean Image")
plt.show()


In [ ]:
# Check if the mean of the columns in X is close to zero
if np.allclose(np.mean(X, axis=1), 0, atol=1e-6):
    print("Part 2: Pass - Data matrix X is correctly mean-centered.")
else:
    print("Part 2: Fail - Data matrix X is not mean-centered properly.")

## Part 3: Compute SVD
Compute the SVD of $X$. Plot the singular values. How many principal components are needed to capture $70\%$ of the training image data? $80\%$? $90\%$? $95\%$? By what factor can we reduce the storage required to store each image if we only need to recover $90\%$ of the data in the image?

In [ ]:
# *************************
# Implement your code here
# 
# *************************

In [ ]:
# Check the shape of U, s, and Vh
if U.shape == (img_size, num_imgs_total) and s.shape == (num_imgs_total,) and Vh.shape == (num_imgs_total, num_imgs_total):
    print("Part 3: Pass - SVD computation completed with correct matrix dimensions.")
else:
    print("Part 3: Fail - SVD matrix dimensions do not match expectations.")

# Check if the cumulative variance sums to 1 (within numerical tolerance)
singular_values_squared = s ** 2
total_variance = np.sum(singular_values_squared)
cumulative_variance = np.cumsum(singular_values_squared) / total_variance

# Check if the cumulative variance sums to 1 within a numerical tolerance
if np.isclose(cumulative_variance[-1], 1.0, atol=1e-6):
    print("Part 3: Pass - Cumulative variance is correctly computed.")
else:
    print("Part 3: Fail - Cumulative variance computation may have issues.")


## Part 4: Approximation error calculations
Suppose that we represent each image using the first $d=20$ features identified with PCA. What is the average approximation error of this representation? Repeat this step with $d=50,70,100$. Pick one of the training images and plot the original image along with the reconstruction for each value of $d$.

In [ ]:
# *************************
# Implement your code here
# 
# *************************

In [ ]:
for d in d_values:
    U_d = U[:, :d]  # Select the first d columns of U
    S_d = np.diag(s[:d])  # Create a diagonal matrix of the first d singular values
    Vh_d = Vh[:d, :]  # Select the first d rows of Vh

    # Approximate the data matrix using the top d components
    X_approx = U_d @ S_d @ Vh_d

    # Compute the average approximation error
    error = np.linalg.norm(X - X_approx, 'fro') ** 2
    average_approximation_error.append(error / num_imgs_total)

    # Reconstruction for a sample image (e.g., first image)
    sample_image_index = 0
    reconstructed_image = X_approx[:, sample_image_index].reshape(img_height, img_width)

    # Plot the original and reconstructed images
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(img_matrix[:, sample_image_index].reshape(img_height, img_width), cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(reconstructed_image, cmap='gray')
    plt.title(f'Reconstructed Image (d={d})')

    plt.show()


In [ ]:
# *************************
# Implement your code here
# 
# *************************

print("Updated average approximation errors for each d:", average_approximation_error)
tolerance = 1e-5
if all(earlier >= later - tolerance for earlier, later in zip(average_approximation_error, average_approximation_error[1:])):
    print("Part 4: Pass - Approximation error decreases as d increases (within tolerance).")
else:
    print("Part 4: Fail - Approximation error does not decrease as expected.")


## Part 5: Reconstruct images
Load the images from the remaining two subjects. Using the first 50 features from the training dataset, compute the feature values for all of the images in the test dataset. What is the approximation error?

In [ ]:
# *************************
# Implement your code here
# 
# *************************

# Calculate the average approximation error for the test set
average_approximation_error_test_set = np.mean(approximation_errors)
print(f"Average approximation error for the test set with d={d}: {average_approximation_error_test_set}")

# Visualize a sample reconstructed image
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(test_img_matrix[:, 0].reshape(img_height, img_width), cmap='gray')
plt.title('Original Test Image')

plt.subplot(1, 2, 2)
plt.imshow(reconstructed_image.reshape(img_height, img_width), cmap='gray')
plt.title(f'Reconstructed Image (d={d})')
plt.show()


## Part 6: Reconstruct rotated image
Repeat the previous step using the rotated image `subject15rotated.jpeg'.

In [ ]:
rotated_img_data = imageio.imread('subject15rotated.jpeg')
# *************************
# Implement your code here
# 
# *************************

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(rotated_img_data.reshape(img_height, img_width), cmap='gray')
plt.title('Original Rotated Image')

plt.subplot(1, 2, 2)
plt.imshow((img_vector - feature_mean).reshape(img_height, img_width), cmap='gray')
plt.title(f'Reconstructed Rotated Image (d={d})')

plt.show()
